In [217]:
import sys
import ctypes
import pandas as pd
import os
import json
import time
from tqdm import tqdm
from datetime import datetime
import ast
import warnings
import numpy as np
warnings.filterwarnings(action='ignore') 

In [218]:
PATH = '/Users/jungyulyang/programming/stair_to_heaven/'

In [219]:
#분봉 데이터 파일들을 모두 리스트에 넣기
file_list = os.listdir(PATH + "catch_highest/data/minute_stock_data/")
file_list.sort()
if '.DS_Store' in file_list:
    file_list = file_list[1:]
else:
    None

In [220]:
#처음날짜
for i in range(len(file_list)):
    if file_list[i].startswith(str(202009)):
        file_list = file_list[i:]
        break
#끝날짜
for i in reversed(range(len(file_list))):
    if file_list[i].startswith(str(202011)):
        file_list = file_list[:i+1]
        break


In [221]:
single = False
# 단일가일때는 다르게 불러옴
if single:
    df = pd.read_csv(PATH + "catch_highest/data/extracted_data/only_time_single_price.csv", usecols = ['time'])
else:
    df = pd.read_csv(PATH + "catch_highest/data/extracted_data/only_time.csv", usecols = ['time'])

In [222]:
file_list

['20200901_한화투자증권우.csv',
 '20200902_쌍용양회우.csv',
 '20200902_일양약품우.csv',
 '20200902_코오롱글로벌우.csv',
 '20200903_한화투자증권우.csv',
 '20200904_신풍제약우.csv',
 '20200904_쌍용양회우.csv',
 '20200904_한화솔루션우.csv',
 '20200907_쌍용양회우.csv',
 '20200907_한화솔루션우.csv',
 '20200908_두산2우B.csv',
 '20200908_두산솔루스1우.csv',
 '20200908_두산솔루스2우B.csv',
 '20200908_두산퓨얼셀1우.csv',
 '20200908_두산퓨얼셀2우B.csv',
 '20200908_한국금융지주우.csv',
 '20200908_한화솔루션우.csv',
 '20200909_DB하이텍1우.csv',
 '20200910_DB하이텍1우.csv',
 '20200910_한화솔루션우.csv',
 '20200911_DB하이텍1우.csv',
 '20200911_KG동부제철우.csv',
 '20200914_DB하이텍1우.csv',
 '20200914_동부건설우.csv',
 '20200914_신원우.csv',
 '20200914_현대비앤지스틸우.csv',
 '20200915_KG동부제철우.csv',
 '20200915_금강공업우.csv',
 '20200915_동양2우B.csv',
 '20200915_동양3우B.csv',
 '20200915_신원우.csv',
 '20200916_금가

In [223]:
for file in tqdm(file_list):
    file = file[0:-4]
    test = pd.read_csv(PATH + "catch_highest/data/minute_stock_data/%s.csv" % file, usecols=['time','open','low'])
    test.columns = ['time',file,'low']
    test[file] = test[file].astype('float')

    if single and len(test) < 20: 
        pass
    else:
        if len(test) < 20:
            continue
        if len(str(test[file][0])) == 3:
            continue
   
    temp = pd.merge(df,test, on='time', how = 'left')

    # 중간중간 vi 걸린 것들은 다음값으로 채워주고
    for i in range(temp.shape[0]):
        if np.isnan(temp[file][i]):
            index = i
            stopping = True
            try:
                while stopping:
                    if np.isnan(temp[file][index+1]):
                        index += 1
                    else:
                        valid = temp[file][index+1]
                        stopping = False
                temp[file][i] = valid
            except:
                continue
        
        if np.isnan(temp['low'][i]):
            index = i
            stopping = True
            try:
                while stopping:
                    if np.isnan(temp['low'][index+1]):
                        index += 1
                    else:
                        valid = temp['low'][index+1]
                        stopping = False
                temp['low'][i] = valid
            except:
                continue

    for j in range(1,temp.shape[0]):
        open_value = round((temp[file][j] - temp[file][0]) / temp[file][0],4) - 0.0025
        low_value = round((temp['low'][j] - temp[file][0]) / temp[file][0],4) - 0.0025

        if low_value <= -0.05:
            temp[file][j:] = low_value
            break
        else:
            temp[file][j] = open_value

    temp = temp[['time',file]]
    df = pd.merge(df, temp, on='time', how='left')

100%|██████████| 76/76 [00:03<00:00, 20.11it/s]


In [224]:
df

,time,20200901_한화투자증권우,20200902_쌍용양회우,20200902_일양약품우,20200902_코오롱글로벌우,20200903_한화투자증권우,20200904_신풍제약우,20200904_쌍용양회우,20200904_한화솔루션우,20200907_쌍용양회우,...,20201027_삼성물산우B,20201027_호텔신라우,20201029_쌍용양회우,20201029_한양증권우,20201030_쌍용양회우,20201102_대호피앤씨우,20201103_덕성우,20201104_덕성우,20201112_덕성우,20201117_대한항공우
0,901,5300.0000,23350.0000,125500.0000,59600.0000,6300.0000,244500.0000,33800.0000,41700.0000,41500.0000,...,128000.0000,91400.0000,47350.0000,25800.0000,59200.0000,2980.0000,14000.0000,15650.0000,20050.0000,38100.0000
1,902,-0.0025,-0.0025,-0.0543,0.0042,-0.0025,0.0159,-0.0025,0.0095,-0.0025,...,0.0287,0.0741,-0.0025,0.0615,-0.0025,-0.1015,-0.0025,-0.0696,-0.0025,-0.0419
2,903,-0.0025,-0.0025,-0.0543,-0.0344,-0.0025,0.0384,-0.0025,0.0203,-0.0025,...,0.0717,0.1080,-0.0025,0.1099,-0.0025,-0.1015,-0.0025,-0.0696,-0.0025,0.0841
3,904,-0.0968,0.0211,-0.0543,0.0378,-0.0708,0.0834,-0.0025,0.0527,-0.0676,...,0.0991,0.1080,0.0133,0.1099,0.0195,-0.1015,0.0082,-0.0696,0.0199,0.1235
4,905,-0.0968,0.0489,-0.0543,0.1301,-0.0708,0.1018,-0.0025,0.1342,-0.0676,...,0.0991,0.0850,0.0746,0.1448,0.0769,-0.1015,-0.0561,-0.0696,0.0299,0.1235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,1517,-0.0968,-0.1010,-0.0543,0.0394,-0.0708,-0.0863,-0.0025,0.2085,-0.0676,...,-0.0611,-0.0605,0.0746,0.0537,-0.0532,-0.1015,-0.0561,-0.0696,-0.0524,-0.0576
377,1518,-0.0968,-0.1010,-0.0543,0.0378,-0.0708,-0.0863,NaN,0.2085,-0.0676,...,-0.0611,-0.0605,0.0746,0.0440,-0.0532,-0.1015,-0.0561,-0.0696,-0.0524,-0.0576
378,1519,-0.0968,-0.1010,-0.0543,0.0260,-0.0708,-0.0863,NaN,0.2085,-0.0676,...,-0.0611,-0.0605,0.0746,0.0518,-0.0532,-0.1015,-0.0561,-0.0696,-0.0524,-0.0576
379,1520,-0.0968,-0.1010,-0.0543,0.0243,-0.0708,-0.0863,NaN,0.2085,-0.0676,...,-0.0611,-0.0605,0.0746,0.0479,-0.0532,-0.1015,-0.0561,-0.0696,-0.0524,-0.0576


In [214]:

#해당 날짜에 값이 없는 경우에는 모두 0으로 대체
df = df.fillna(0)

#기준이 되는 0901 지우고 데이터 행-열 변환
df = df.iloc[1:,:].T.reset_index()
df.columns = df.iloc[0]
df = df.iloc[1:,:].reset_index(drop=True)

#날짜와 기업 자르고 다시 재정렬
df['date'] = [str() for x in range(len(df.index))]
df['company'] = [str() for x in range(len(df.index))]




In [215]:
df = df.drop(df.columns[[-1,-2]],axis=1)

In [216]:
df.to_csv("test.csv",encoding='utf-8-sig')